# ベースライン指標

In [85]:
import joblib
train_df, test_df = joblib.load("../data/train_test_split.pkl")

In [93]:
from surprise import Dataset, Reader, BaselineOnly, accuracy

In [89]:
reader = Reader(rating_scale=(1, 10))
data_train = Dataset.load_from_df(train_df, reader)
data_test = [tuple(e) for e in zip(test_df["user_id"], test_df["book_id"], test_df["book_rating"])]
full_data = data_train.build_full_trainset()

model = BaselineOnly()


In [91]:
model.fit(full_data)
predictions = model.test(data_test)

Estimating biases using als...


In [97]:
from collections import defaultdict

def precision_recall_at_k(
        predictions,
        k: int = 5,
        threshold: int = 8
    ):
        """Return precision and recall at k metrics for each user"""

        user_est_true = defaultdict(list)
        for uid, _, true_r, est, _ in predictions:
            user_est_true[uid].append((est, true_r))

        precisions = dict()
        recalls = dict()
        for uid, user_ratings in user_est_true.items():

            user_ratings.sort(key=lambda x: x[0], reverse=True)
            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
            n_rel_and_rec_k = sum(
                ((true_r >= threshold) and (est >= threshold))
                for (est, true_r) in user_ratings[:k]
            )
            precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
            recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        precision = sum(prec for prec in precisions.values()) / len(precisions)
        recall = sum(rec for rec in recalls.values()) / len(recalls)

        return precision, recall


In [98]:
rmse = accuracy.rmse(predictions, verbose=False)
precision, recall = precision_recall_at_k(predictions)

In [99]:
print(f"RMSE: {rmse:.2}")
print(f'precision: {precision:.2f}')
print(f"recall: {recall:.2f}")

RMSE: 1.6
precision: 0.52
recall: 0.50


In [100]:
# 評価値の中央値、平均値が8.0なので、precision, recallが0.5になるのは納得